<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Open Source Decision Intelligence

Decision Intelligence applied in this module:  
* Listing key factors to consider when making a sound decision  
* Decision Scenario: Use a decision framework (Ben Franklin's Pro & Con List) to create a decision plan  
* Improving Decision Intelligence process by explicitly proving decision frameworks and additional context 

A recommended enterprise pattern is to scale Articial Intelligence strategy with three key areas:
* Commercial AI (OpenAI and other proprietary Generative AI providers)
* Open-Source AI (open-source AI providers)
* Vendor and Partner AI (i.e. company HR Software, contract software) 

These three areas together strategically form AI capability and capacity in what I like to refer to as the "Generative AI Brain". This is illustrated below with sample providers. For example, an organization may have an existing relationship with Adobe for their graphic design and leverage the built-in Generative AI capabilities rather building their own. 

<img style="display: block; margin: auto;" width ="800px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/AIBrainPillars.png">

Semantic Kernel embraces AI orchestration across all the pillars mentioned above. It allows all types of models (commercial or proprietary) and almost any APIs to be orchestrated to faciliate enterprise Decision Intelligence.

### Step 1 - Get Started with LMStudio and Local Open Source AI Models 

This module highlights how to use local GenAI models (i.e. LLMs) with Semantic Kernel. To illustrate this the Phi-4 model is run locally using LMStudio as a local REST endpoint that will interface with Semantic Kernel orchestration.  

Phi-4 is considered an SLM (Small Language Model) with parameters generally below the 27 Billion parameter threshhold. This doesn't mean the model is not capable of performing well. While it certainly may lack the breadth of general knowledge of Large Language Models, Phi-4 makes up for it by providing very capable reasoning capabilities that can rival LLMs. Below is an image illustrating this performance:  
<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Phi4-Performance.webp">   

Steps to get started:
* Download & install LMStudio: https://lmstudio.ai/ (Windows, Mac or Linux) 
* Run the LMStudio studio application 
* In the LMStudio application, download the Phi-4 GGUF file. Search for "Phi-4" in the "Discover" section of LMStudio. A variety of Phi-4 options that are official and unofficial from hobbyists will appear. Typically, selecting the model with the most downloads will provide the best results.  
* In the experiment below, the 14B parameter model is being used. LMStudio will inspect your hardware and let you know which quantized version of the model(s) is optimal for your hardware. Note: Even computers with small graphics cards can run these models well locally. Furthermore, laptops such as the Macbook Pro with Neural Engine can run LMStudio local models as well. 
* Start the LMStudio Server with the Phi-4 model loaded. This will start a local REST endpoint with a URI similar to http://localhost:1234/v1 
* The LMStudio local server does not have default security, you can simply check by navigating to this link in any browser to check if a model is loaded: http://localhost:1234/v1/models 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/LMStudioServer.png"> 

### Step 2 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next cell to:
* Use the Configuration Builder to use the local LMStudio Server  
* Use the API configuration to build the Semantic Kernel orchestrator  
* Notice there is no security being passed in and it is simply a URL

In [1]:
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.45"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#pragma warning disable SKEXP0010 
var semanticKernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(
        modelId: "phi4-14b-thinking",
        endpoint: new Uri("http://localhost:1234/v1/"),
        apiKey: null)
    .Build();
#pragma warning restore SKEXP0010

// Retrieve the Chat Completion Service
var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.0 Microsoft.SemanticKernel, 1.45.0

### Step 3 - Open Source AI Models with Decision Intelligence 

Semantic Kernel allows one to interact with any API service that adheres to the OpenAI specification. Notice the method to add LMStudio capability was simply enabled via the **AddOpenAIChatCompletion** method.

Execute the cell below about decision factors for a investment property. Note:
* OpenAI Prompt Execution Settings are the same in LMStudio as they are for OpenAI and Azure OpenAI
* Passing in arguents works the same way in Semantic Kernel
* Streaming works as well in Semantic Kernel for supported services

> 📜 "Don't wait to buy real estate. Buy real estate and wait."  
>
> -- <cite>Will Rodgers (American humorist, actor, and social commentator in the early 20th century)</cite>  

In [2]:
// Prompt Engineering works in a very similar way to native OpenAI API models

var systemDecisionPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Create a Decision Intelligence prompt on the topic of purchasing a secondary home as an investment property
// Provide detailed decision-making criteria for evaluating the investment decision
var simpleDecisionPrompt = """
You are considering purchasing a secondary home as an investment property. 

What key factors should you evaluate to ensure a sound investment decision, including financial, market, and property-specific considerations? 
Outline the critical steps and criteria for assessing location, potential rental income, financing options, long-term property value, and associated risks.
""";

// You can set the typical OpenAI settings with most open-source models
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
// Note: The settings below are similar to OpenAI API settings. Most OSS models support these settings  
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemDecisionPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings))
{
    chatResponse += content;
    Console.Write(content);
}

// Add the assistant's response to the chat history
// This allows you to maintain the context of the conversation for future messages
chatHistory.AddAssistantMessage(chatResponse);

When considering purchasing a secondary home as an investment property, it's crucial to conduct thorough research and analysis across several dimensions. Here’s a structured approach to evaluating this decision:

### 1. **Location Analysis**

- **Market Demand**: Research the local housing market demand for rentals. Look at population growth trends, employment rates, and economic drivers in the area.
  
- **Neighborhood Quality**: Consider factors such as crime rates, quality of schools, proximity to amenities (parks, shopping centers), and transportation links.

- **Future Development Plans**: Investigate any planned infrastructure projects or developments that could affect property values positively or negatively.

### 2. **Potential Rental Income**

- **Comparable Rentals**: Analyze rental prices for similar properties in the area. Websites like Zillow, Rentometer, or local real estate listings can provide insights into current market rates.

- **Occupancy Rates**: Evaluate historic

Advanced Prompt Engineering techniques can be applied to OSS (open-source) models as well. In the example below a more advanced reasoning decision prompt will be used to provide additional instructions to the GenAI model.

In [3]:
// Prompt Engineering works in a very similar way to native OpenAI API models

var systemDecisionPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Create a Decision Intelligence prompt on the topic of purchasing a secondary home as an investment property
// Use Chain of Thought to prompt the OSS model
// Use the Minto Pyramid to communicate the decision 
var advancedDecisionPrompt = """
You are considering purchasing a secondary home as an investment property. 

Before providing any answer, you must: 
Understand the Problem: Carefully read and understand the user's question or request. 
Break Down the Reasoning Process: Outline the steps required to solve the problem or respond to the request logically and sequentially. Think aloud and describe each step in detail. 
Always aim to make your thought process transparent and logical. 
Explain Each Step: Provide reasoning or calculations for each step, explaining how you arrive at each part of your answer. 
Provide structured, logical, and comprehensive advice. 
Arrive at the Final Answer: Only after completing all steps, provide the final answer or solution. 
Review the Thought Process: Double-check the reasoning for errors or gaps before finalizing your response. 
Communicate the final decision using the Minto Pyramid Principle.
""";

// You can set the typical OpenAI settings with most open-source models
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
// Note: The settings below are similar to OpenAI API settings. Most OSS models support these settings  
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// Create a new chat history object with proper system and user message roles
var chatHistoryAdvancedDecisionPrompt = new ChatHistory();
chatHistoryAdvancedDecisionPrompt.AddSystemMessage(systemDecisionPrompt);
chatHistoryAdvancedDecisionPrompt.AddUserMessage(advancedDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryAdvancedDecisionPrompt, openAIPromptExecutionSettings))
{
    chatResponse += content;
    Console.Write(content);
}

// Add the assistant's response to the chat history
// This allows you to maintain the context of the conversation for future messages
chatHistoryAdvancedDecisionPrompt.AddAssistantMessage(chatResponse);

To assist you in deciding whether to purchase a secondary home as an investment property, we need to systematically evaluate various factors and considerations. Here's how we can approach this decision:

### Understand the Problem
You are considering purchasing a secondary home for investment purposes. The goal is to determine if this is a wise financial decision based on current market conditions, personal circumstances, and long-term objectives.

### Break Down the Reasoning Process

1. **Define Objectives:**
   - Clarify why you want to purchase an investment property. Are you looking for rental income, capital appreciation, tax benefits, or diversification of your portfolio?

2. **Market Research:**
   - Analyze current real estate market trends in potential locations.
   - Consider factors such as demand and supply dynamics, average rental yields, property price growth rates, and economic indicators.

3. **Financial Analysis:**
   - Assess your financial situation to determine aff

### Step 4 - Open Source AI Models with Ben Franklin Decision Framework 

> 📜 "By failing to prepare, you are preparing to fail." 
>
> -- <cite>Ben Franklin (Founding Father of the United States, inventor, godfather of Decision Science)</cite> 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/Articles/main/20230326-Make-Great-Decisions-Using-Ben-Franklins-Pros-And-Cons-Method/Image-BenFranklinDecisionMakingMethod.png">

#### Tom Brady's use of a Decision Framework

Tom Brady's decision to join the Tampa Bay Buccaneers in 2020 marked a significant in his legendary NFL career. After 20 seasons and six Super Bowl championships with the New England Patriots, Brady became a free agent and chose to sign with the Bucs. How did he arrive at this decision? On the Fox broadcast on 09.29.2024, while covering the Buccaneers vs Philadelphia Eagles game, Tom Brady described how he arrived at this decision.

In the screenshot below, Tom Brady is holding up some small paper cards he is showing the audience of the broadcast. Brady mentioned he wrote down the personal decision criteria that was important and how each team compared in that criteria (salary, weather etc). He used this to select the Tampa Bay Buccaneers as his team, where he went on to win a Super Bowl in his first year there! **After 250 years since it's inception, Tom Brady used the "Ben Franklin Decision Framework" to decide where to play NFL quaterback!!**  

> 📜 "There was a lot of reasons to choose Tampa, and I made about 18 criteria why, and there was things all the way from salary, obviously, to the weather to the facilities to how great the players were..."
>
> -- <cite>Tom Brady (7-time Super Bowl Champion, Hall of Fame NFL Quaterback)</cite> 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/OpenSourceDecisionIntelligence-BenFranklinDecisionFramework-TomBrady.png">

#### Steps for Ben Franklin's Decision Framework

Below are the steps Ben Franklin recommends when making a decision, which he called his "Decision Making Method of Moral Algebra":  
- Frame a decision that has two options (Yes or a No)
- Divide an area into two competing halves: a "Pro" side and "Con" side
- Label the top of one side "Pro" (for) and the other "Con" (against)
- Under each respective side, over a period of time (Ben Franklin recommended days, this could be minutes) write down various reasons/arguments that support (Pro) or are against (Con) the decision
- After spending some time thinking exhaustively and writing down the reasons, weight the different Pro and Con reasons/arguments
- Determine the relative importance of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- The side with the most remaining reasons is the option one should select for the decision in question

Learn more about Ben Franklin's Decision Framework: https://medium.com/@bartczernicki/make-great-decisions-using-ben-franklins-decision-making-method-c7fb8b17905c  

#### Decision Scenario - Should a Family Take a Luxury Vacation

Should a family take a luxury family vacation this year? Just like Brady mapped out whether joining the Bucs would satisfy his key personal and professional goals, you can list the factors that matter most for your family—budget, timing, destination climate, activities for the kids—and lay them out on your own “decision cards.” Weigh each component carefully, just as Brady weighed his NFL future. Because if it worked to land Brady in Tampa (where he won yet another Super Bowl), imagine what it can do for a family’s dream getaway.

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-Vacation.png">

In [ ]:
// You can also use a template to provide more context to the model combining system and user prompts
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Your role is to guide users in exploring options, analyzing decisions, solving complex problems, and applying systems thinking to diverse scenarios. 
Provide structured, logical, and comprehensive advice, ensuring clarity, depth, and actionable insights to support informed decision-making.
""";
var benFranklinDecisionPrompt = """
Apply the Ben Franklin Decision-Making Framework (Pro and Con list) to evaluate whether or not to take a luxury family vacation. 
List at most 10 pros and at most 10 cons to help the user make an informed decision.
""";

var simpleDecisionPromptTemplate = $"""
{systemDecisionPrompt}
--------------
{benFranklinDecisionPrompt}
""";

// You can set the typical OpenAI settings with most open-source models
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.2, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// Stream the response output 
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptTemplate, kernelArguments))
{
   Console.Write(streamChunk);
}

The Ben Franklin Decision-Making Framework is a classic method for evaluating options by listing their advantages and disadvantages. Here's how you can apply this framework to decide whether or not to take a luxury family vacation:

### Pros of Taking a Luxury Family Vacation

1. **Quality Time Together**: Provides an opportunity for the family to bond, create memories, and strengthen relationships.
2. **Stress Relief**: A break from daily routines can reduce stress and improve mental health for all family members.
3. **Cultural Exposure**: Offers exposure to new cultures, languages, and experiences that can be educational and enriching.
4. **Adventure and Fun**: Provides excitement and adventure, which can be especially beneficial for children’s development and happiness.
5. **Health Benefits**: Relaxation and change of scenery can improve physical health by reducing stress-related ailments.
6. **Memorable Experiences**: Creates lasting memories that the family can cherish for years t

#### Improving the Ben Franklin's Decision Framework with SLMs

Notice that the output above may not be exactly what most would anticipate. The Ben Franklin framework was not fully understood nor applied. Open-Source GenAI models that have a small amount of parameters (< 13 billion parameters) may not have all the inherent knowledge "trained" into the model. The exception being domain-specific models that are only trained on data sets for that domain. Those models can have a great deal of knowledge, while maintaining a small amount of parameters. 

One simple way to improve the outcome is to provide the explicit steps of the "Ben Franklin Decision Framework" into the prompt context. This basically provides the instructions of the decision framework directly to the model; regardless if the GenAI model was trained with decision framework data.  

In the example below, the prompt context is provided with the Ben Franklin Decision Framework steps. Contrast this with the example above, where the decision recommendation is not clear. 

In [6]:
// You can also use a template to provide more context to the model combining system and user prompts
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Your role is to guide users in exploring options, analyzing decisions, solving complex problems, and applying systems thinking to diverse scenarios. 
Provide structured, logical, and comprehensive advice, ensuring clarity, depth, and actionable insights to support informed decision-making.
""";

var benFranklinDecisionPrompt = """
Apply the following steps IN ORDER of the Ben Franklin Decision Framework to the Question below:
- Frame a decision that has two options (Yes or a No)
- Divide an area into two competing halves: a "Pro" side and "Con" side
- Label the top of one side "Pro" (for) and the other "Con" (against)
- Under each respective side, list a maximum of 10 reasons or arguments for each option
- Consider the weight of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- Determine the relative importance of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- The side with the most remaining reasons is the option one should select for the decision in question
- IMPORTANT: ALWAYS recommend a decision based on the side with the most reasons, even if the reasons are of lesser value than the other side!

Question: Should I take a luxury family vacation?
""";

var simpleDecisionPromptTemplate = $"""
{systemDecisionPrompt}
--------------
{benFranklinDecisionPrompt}
""";

// You can set the typical OpenAI settings with most open-source models
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptTemplate, kernelArguments))
{
   Console.Write(streamChunk);
}

To apply the Ben Franklin Decision Framework to your question about whether you should take a luxury family vacation, let's follow each step systematically.

### Frame the Decision
**Decision:** Should I take a luxury family vacation?

- **Option 1 (Yes):** Take the luxury family vacation.
- **Option 2 (No):** Do not take the luxury family vacation.

### Divide into Pro and Con Sides

#### Pros of Taking the Luxury Family Vacation:
1. **Memorable Experience:** Provides lasting memories for the entire family.
2. **Stress Relief:** Offers a break from daily routines, reducing stress levels.
3. **Quality Time:** Enhances bonding time with family members.
4. **Cultural Exposure:** Opportunity to experience new cultures and environments.
5. **Health Benefits:** Potential health benefits from relaxation and possibly better nutrition.
6. **Adventure and Fun:** Provides excitement and adventure for all ages.
7. **Skill Development:** Children can learn new skills or hobbies during the trip.
8.

The GenAI model may or may not recommend a luxury vacation, but this can be dramatically improved further! Imagine if the GenAI model had access to: your finances, current stress level, the last time you took a vacation, any upcoming major purchases, family dynamic?!  

In the example below, additional context is provided with that information. Notice how it changes the the Pro and Con list.  

Just like Tom Brady, it could craft a Pro and Con list specific and personalized to your scenario!

In [7]:
// You can also use a template to provide more context to the model combining system and user prompts
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Your role is to guide users in exploring options, analyzing decisions, solving complex problems, and applying systems thinking to diverse scenarios. 
Provide structured, logical, and comprehensive advice, ensuring clarity, depth, and actionable insights to support informed decision-making.
""";
// Try changing the background information to see how it affects the decision-making process
var backroundInformation = """
BACKGROUND INFORMATION OF FAMILY: 
You are considering purchasing a secondary home as an investment property. 
You have been stressed out at work. 
You have been working long hours and have not taken a vacation in over a year. 
Your kids are about to enter college and they don't have scholarships. 
Your car is finishing its lease and will need to be replaced soon. 
""";
// Try to adjust the specificity of the decision-making criteria to see how it affects the decision-making process
var benFranklinDecisionPrompt = """
Apply the following steps IN ORDER of the Ben Franklin Decision Framework to the Question below:
- Frame a decision that has two options (Yes or a No)
- Divide an area into two competing halves: a "Pro" side and "Con" side
- Label the top of one side "Pro" (for) and the other "Con" (against)
- Under each respective side, list the relevvant reasons from "BACKGROUND INFORMATION OF FAMILY" section and or create new reasons that are ONLY closely related to the "BACKGROUND INFORMATION OF FAMILY". 
- Consider the weight of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- Determine the relative importance of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- The side with the most remaining reasons is the option one should select for the decision in question
- IMPORTANT: ALWAYS recommend a decision based on the side with the most reasons, even if the reasons are of lesser value than the other side!

Question: Should I take a luxury family vacation?
""";

var simpleDecisionPromptTemplate = $"""
{systemDecisionPrompt}
--------------
{backroundInformation}
--------------
{benFranklinDecisionPrompt}
""";

// You can set the typical OpenAI settings with most open-source models
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptTemplate, kernelArguments))
{
   Console.Write(streamChunk);
}

To apply the Ben Franklin Decision Framework to your question about whether you should take a luxury family vacation, let's follow each step methodically:

### Step 1: Frame the Decision
**Decision:** Should you take a luxury family vacation?

- **Option Yes:** Take the luxury family vacation.
- **Option No:** Do not take the luxury family vacation.

### Step 2: Divide into Pro and Con Sides

#### Pro Side (For taking the vacation)
- **Relieve Stress:** You have been stressed out at work, and a vacation could provide much-needed relaxation and mental health benefits.
- **Family Bonding:** A vacation can strengthen family bonds, especially with your kids about to enter college—a significant life transition for them.
- **Work-Life Balance:** Taking time off after working long hours without a vacation in over a year could improve your overall work-life balance.

#### Con Side (Against taking the vacation)
- **Financial Strain:** You are considering purchasing an investment property, which

Notice how providing personal family background information changes the entire dynamic of the information used in the decision framework and how it influences the recommended decision. The decision process is more specific not only to the scenario, but also it provides contextual background information. This makes the decision process more personalized and potentially much more accurate! 

### Step 5 - Open Source AI Models with Commercial AI Providers  

Semantic Kernel can include mutliple AI service providers. This allows for hybrid workflows from a single Semantic Kernel instance like: 
* Capability Optimizations: Use SLMs for domain specific tasks and LLMs for complex decision reasoning
* Decision Optimizations: Apply an (ensemble) decision self-consitency pattern  
* Capacity Optimizations: Splitting functions, plugins, personas or agents across different AI services

In [ ]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.39.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// Retrieve the configuration settings for the Azure OpenAI service
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];

// Example to build a Kernel with Azure OpenAI and Opensource AI
#pragma warning disable SKEXP0010 
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        modelId: azureOpenAIModelDeploymentName,
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .AddOpenAIChatCompletion(
        modelId: "Phi-4",
        endpoint: new Uri("http://localhost:1234/v1/"),
        apiKey: "LMStudio")
    .Build();
#pragma warning restore SKEXP0010

// var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings {
//     MaxTokens = 1500,
//     Temperature = 0.3, 
//     TopP = 1.0, 
//     FrequencyPenalty = 0.0, 
//     PresencePenalty = 0.0
//     };
// KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0 Microsoft.SemanticKernel, 1.33.0